# Independent Component Analysis

DataFun

Members:
Jan Botscch
Fabian Frank
David Munkacsi

In [1]:
import numpy as np
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import numpy as np

#defines
samplerate = 8192
N = 2
p = 18000

#load the data
dataSet1 = np.loadtxt('./sounds/sound1.dat')
dataSet2 = np.loadtxt('./sounds/sound2.dat')
s = np.stack([dataSet1, dataSet2], axis=0)

print (s.shape)

# create random and invertible NxN (2x2) matrix
while True:
    A = np.random.rand(2,2)
    if np.linalg.det(A) != 0.0:
        break
print("A=" + str(A))

#mix the sources
x = np.matmul(A, s)
print(x[:,:10])
print(x.shape)

# remove temporal structure by permutation 
x_per = x[:, np.random.permutation(range(0,18000))]
print(x_per[:,:10])
print("x_per=" + str(x_per.shape))

#calculate correlation between sources and mixtures
s_std = np.std(s,axis=1)
x_per_std = np.std(x_per,axis=1)
print(s_std)
print(x_per_std) 

corr = np.zeros((2, 2))
for i in range(0,2):
	for j in range(0,2):
		#print("i=" + str(i) + " j=" + str(j))
		corr[i,j] = np.cov(s[i,:], x_per[j,:])[0][1] / (s_std[i] * x_per_std[j])
		#print(np.cov(s[i,:], x_per[j,:]))
		#print("corr[i,j]=" + str(corr[i,j]))
print("correlations=" + str(corr))

#center the data
x_per = x_per - np.mean(x_per,axis=0);

#initialize W at random
W = np.random.rand(2,2)
W_nat = np.copy(W)
print("W_nat=" + str(W_nat))
def stepSigmoid(y):
	return 1 - 2 * (1 / (1 + np.exp(-y)))
	
vStepSigmoid = np.vectorize(stepSigmoid)
eps = 0.01
for i in range(0,1):
	W_inv = np.linalg.inv(W) 
#	print("W_inv=" + str(W_inv)) 
#	currX = x_per[:,i]
#	print("currX=" + str(currX)) 
#	inpSig = np.dot(W,x_per[:,i])
#	print("inpSig=" + str(inpSig)) 
#	print("inpSigShape=" + str(inpSig.shape)) 
#	vStepSigmoidRes = vStepSigmoid(np.dot(W,x_per[:,i]))
#	print("vStepSigmoidRes=" + str(vStepSigmoidRes)) 
#	print("vStepSigmoidResShape=" + str(vStepSigmoidRes.shape)) 
#	outer = np.outer(vStepSigmoid(np.dot(W,x_per[:,i])),x_per[:,i])
#	print("outer=" + str(outer)) 
#	print("outerShape=" + str(outer.shape)) 
	dW = W_inv.T + np.outer(vStepSigmoid(np.dot(W,x_per[:,i])),x_per[:,i])
#	print("at iter" + str(i) + ", dW=" + str(dW))
	W = W + eps * dW
#	print("at iter" + str(i) + ", W=" + str(W))
	#W_nat_inv = np.linalg.inv(W_nat) 
	#dW_nat = W_nat_inv.T + np.outer(vStepSigmoid(np.dot(W_nat,x_per[:,i])),x_per[:,i])
	#W_nat = W_nat + eps * dW_nat * (np.matmul(W_nat.T, W_nat))

print("final=" + str(A - np.linalg.inv(W)))
#print("final nat=" + str(A - np.linalg.inv(W_nat)))

(2, 18000)
A=[[ 0.00421397  0.8770905 ]
 [ 0.8016555   0.95779517]]
[[ 0.         -0.00013945 -0.00339928 -0.00212185  0.0005594   0.0043103
   0.00255526 -0.0039623  -0.00566209 -0.01529176]
 [ 0.         -0.00015228 -0.00374215 -0.00306176  0.00091174  0.00395471
   0.00143644 -0.00369505 -0.00889097 -0.01588645]]
(2, 18000)
[[ 1.01834208  0.76214846  0.05319465 -1.19212751 -0.27936371  0.02776662
  -0.57234043 -0.31052535  2.32930082  0.7528102 ]
 [-0.15579499  0.97148466  0.0553062  -1.41109068  0.32810189  0.03331526
  -1.60544589 -0.35107292  2.53764186 -0.80500547]]
x_per=(2, 18000)
[ 0.99854594  0.99885435]
[ 0.87610106  1.24819281]
correlations=[[ 0.01469677  0.00782925]
 [-0.00077738  0.00256299]]
W_nat=[[ 0.65710988  0.92029467]
 [ 0.41719034  0.16251409]]
final=[[ 0.42626385 -1.95661622]
 [-0.56218682  2.93312187]]


## 5.3 Results

(a) Plot and Play

In [5]:
# Original sounds.
wav.write('Source1.wav',8192,dataSet1)
wav.write('Source2.wav',8192,dataSet2)

# Mixed sounds.
wav.write('Mixed1.wav',8192,x[0,:])
wav.write('Mixed2.wav',8192,x[1,:])

# Mixed and permutated.
wav.write('MixedPerm1.wav',8192,x_per[0,:])
wav.write('MixedPerm2.wav',8192,x_per[1,:])

# Unmixed with normal gradient.
wav.write('UnmixedNormGrad1.wav',8192,unmixedNormal[0,:])
wav.write('UnmixedNormGrad2.wav',8192,unmixedNormal[1,:])

# Unmixed with natural gradient.
wav.write('UnmixedNatGrad1.wav',8192,unmixedNatural[0,:])
wav.write('UnmixedNatGrad2.wav',8192,unmixedNatural[1,:])

plt.figure()
plt.plot(dataSet1)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound1')

plt.figure()
plt.plot(dataSet2)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound2')

plt.figure()
plt.plot(x[0,:])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound1 mixed')
plt.figure()
plt.plot(x[1,:])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound2 mixed')

plt.figure()
plt.plot(x_per[0,:])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound1 mixed random')
plt.figure()
plt.plot(x_per[1,:])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound2 mixed random')

plt.figure()
plt.plot(unmixedNormal[0,:])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound1 unmixed normal gradient')
plt.figure()
plt.plot(unmixedNormal[1,:])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound2 unmixed normal gradient')

plt.figure()
plt.plot(unmixedNatural[0,:])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound1 unmixed natural gradient')
plt.figure()
plt.plot(unmixedNatural[1,:])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound2 unmixed natural gradient')

NameError: name 'unmixedNormal' is not defined

(b) Calculate correlations between the true sources and the estimations

(c) Comparing learning speed, data whitening

(d) Plotting density